In [6]:
# Imports de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns      
import missingno as msno
import statsmodels.api as sm
import matplotlib.dates as mdates

from apafib import load_dormir
from scipy import stats
from time import time
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import copy


import warnings
warnings.filterwarnings('ignore')

# Definiciones
RND = 16
MAX_ITER = 5000

sns.set(style="whitegrid", font_scale=1.05)

# Funciones auxiliares
def format_pval(p):
    if p == 0:
        return r"$0$"
    exp = int(np.floor(np.log10(p)))
    if exp >= -3:
        return f"{p:.4f}"                
    else:
        mant = p / (10.0**exp)
        return rf"${mant:.2f}\times10^{{{exp}}}$"
    
# quitar warnings:
import warnings
warnings.filterwarnings("ignore", message="findfont:.*")

init_time = time()

# Práctica de APA: Tennis. Preámbulo.

## Introducción

Autores: Oriol Farrés y Marc Gil

Para la práctica de APA del Q1 25-26, hemos decidido analizar partidos de tennis para a partir de múltiples variables, predecir el ganador de un partido de tenis.

Estamos hablando de un problema de **aprendizaje supervisado**, concretamente un problema de **clasificación**.


## Estructura de la práctica

Tras obtener los datos del repositorio de [Jeff Sackman](https://github.com/JeffSackmann/tennis_atp) y prepararlos un poco, al tener un trabajo tan extenso en el preprocesado de los datos, dividiremos el problema en 2 notebooks:

1. Preprocesado de los datos **01_preprocesado.ipynb**
2. Ajuste a modelos lineales y no lineales **02_ajuste_y_conclusiones.ipynb**



---

Voy a ajustar a una regresión logística sin variables categóricas para utilizar el modelo como baseline y garantizar que es mejor que el 50% (la aleatoriedad).

# 1-Preprocesado de los datos

In [7]:
tennis = pd.read_csv('./data/raw/raw_atp_matches.csv')
tennis.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year,month,day,month_name,tourney_points
0,2011-339,Brisbane,Hard,32,A,20110102,1,104417,1.0,NaN,...,4.0,5.0,5580.0,173.0,309.0,2011,1,2,January,250
1,2011-339,Brisbane,Hard,32,A,20110102,2,103582,NaN,NaN,...,5.0,58.0,835.0,75.0,643.0,2011,1,2,January,250
2,2011-339,Brisbane,Hard,32,A,20110102,3,105051,NaN,Q,...,8.0,196.0,263.0,204.0,243.0,2011,1,2,January,250
3,2011-339,Brisbane,Hard,32,A,20110102,4,104797,8.0,NaN,...,3.0,40.0,1031.0,43.0,975.0,2011,1,2,January,250
4,2011-339,Brisbane,Hard,32,A,20110102,5,103888,4.0,NaN,...,6.0,16.0,1991.0,83.0,600.0,2011,1,2,January,250


In [9]:
tennis.describe().T

,count,mean,std,min,25%,50%,75%,max
draw_size,39541.0,5.746749e+01,42.144138,2.0,32.0,32.0,96.00,128.0
tourney_date,39541.0,2.017440e+07,41143.884302,20110102.0,20140319.0,20170731.0,20210726.00,20241218.0
match_num,39541.0,1.672398e+02,151.260611,1.0,21.0,190.0,283.00,1701.0
winner_id,39541.0,1.177377e+05,29694.476989,100644.0,104607.0,105373.0,109739.00,212721.0
winner_seed,16884.0,7.543592e+00,6.988957,1.0,3.0,5.0,9.00,33.0
winner_ht,39113.0,1.873687e+02,7.376512,3.0,183.0,188.0,193.00,211.0
winner_age,39538.0,2.719211e+01,4.202416,14.9,24.2,27.0,30.10,44.6
loser_id,39541.0,1.177053e+05,29518.033677,100644.0,104655.0,105453.0,111153.00,212970.0
loser_seed,9571.0,9.048062e+00,7.540513,1.0,4.0,7.0,12.00,33.0
loser_ht,38573.0,1.865434e+02,7.220876,3.0,183.0,185.0,191.00,211.0


# Tiempo total de ejecución

In [8]:
print(f"Total Running time {timedelta(seconds=(time() - init_time))}")

Total Running time 0:00:08.183733
